In [ ]:
from pettingzoo import ParallelEnv
import pandas as pd

class CustomEnvironment(ParallelEnv):
    metadata = {
        "name": "custom_environment_v0",
    }

    def __init__(self, player_df:pd.DataFrame, max_rounds=14):
        self.player_df = player_df # DataFrame containing information on players available for season's draft (i.e. name, expected pts, position, etc.)
        self.num_players = len(player_df) # Number of total players available to draft

        self.possible_agents = ["player_1", "player_2"] # two agents drafting
        self.agents = self.possible_agents[:]

        self.draft_board = {} # Draft board that updates as we go so all agents can see what players are available
        self.agent_teams = {agent: [] for agent in self.possible_agents} # Dictionary with each team's drafted players
        self.unavailable_players = [] # players that have already been picked
        self.available_player = self._get_all_players(player_df)


        self.current_round = 0
        self.max_rounds = max_rounds

    def reset(self):
        self.agents = self.possible_agents[:]
        self.draft_board = {}
        self.agent_teams = {agent: [] for agent in self.possible_agents}
        self.draft_picks = []

    def step(self, actions):
        # actions are possible choices

        # add to draft picks, add to player_teams, remove someone from draft board
        rewards = {agent: 0 for agent in self.agents}
        terminated = {agent: False for agent in self.agents}
        truncated = {agent: False for agent in self.agents}
        infos = {agent: {} for agent in self.agents}

        for agent, action in actions.items():
            if action is not None:
                player = action
                if self._draft_player(agent, player):
                    rewards[agent] += 1

        self.current_round += 1
        if self.current_round >= self.max_rounds or not self.available_players:
            for agent in self.agents:
                terminated[agent] = True

        return rewards, terminated, truncated, infos

    def render(self):
        # Display current state
        print(f"Round {self.current_round}:")
        for agent in self.agents:
            print(f"{agent} roster: {self.agent_teams[agent]}")
        print(f"Available players: {self.available_players}")

    def observation_space(self, agent):
        # viewable board
        return self.observation_spaces[agent]

    def action_space(self, agent):
        # possible actions

        # action masking with invalid choice
        return spaces.Discrete()
    
    def _get_all_players(self, df):
        return df['player_name'].to_list()
    
